In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.exceptions import SpotifyException
import pandas as pd
import time

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id = 'fb35e1e6b7c54dc6b625996ae72f9f81',
                                                          client_secret = '72eeda8532044330ba8a17eacdea3e62'))

results = sp.search(q='artist: Drake' , limit = 5, type = 'track')
results



{'tracks': {'href': 'https://api.spotify.com/v1/search?query=artist%3A+Drake&type=track&offset=0&limit=5',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'},
       'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4',
       'id': '3TVXtAsR1Inumwj472S9r4',
       'name': 'Drake',
       'type': 'artist',
       'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/7tYKF4w9nC0nq9CsPZTHyP'},
       'href': 'https://api.spotify.com/v1/artists/7tYKF4w9nC0nq9CsPZTHyP',
       'id': '7tYKF4w9nC0nq9CsPZTHyP',
       'name': 'SZA',
       'type': 'artist',
       'uri': 'spotify:artist:7tYKF4w9nC0nq9CsPZTHyP'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'DO',
     

In [2]:
results['tracks']['items'][0]['album']['artists'][0]['name']

all_artist_names = []

for track in results['tracks']['items']:
    for artist in track['album']['artists']:
        all_artist_names.append(artist['name'])

print(all_artist_names)

['Drake', 'SZA', 'Drake', '21 Savage', 'Drake', 'Drake', 'Drake']


### Song catcher 
- Step one
build function to scrape artists from a playlist, and store them in a list
spotipy used: sp.user_playlist_tracks('spotify', 'playlist Uri')

- step two
build function that takes the list and returns 30 songs from the artist and stores them in a list with the Uris
spotipy used: sp.search(q = 'artist : {artist}, limit = 30, type = 'track')
might have to add a failsafe if artist doesnt have 30 songs

- step three
use Uris for each track Uris to get audio features
spotipy used : sp.audio_features


In [3]:
#Scrape artists from spotify and store in a list
def artists_spot(playlist_id):
    
    all_artist_names = []

    results = sp.user_playlist_tracks('spotify', playlist_id )
    #print("Initial Results:", results)
    tracks = results['items']

    for track in tracks:
        #append artist names to list
        for artist in track['track']['album']['artists']:
            all_artist_names.append(artist['name'])
        
        
    while results['next']:
        #handling pages
        results = sp.next(results)
        tracks = results['items']
        for track in tracks:
            for artist in track['track']['album']['artists']:
                all_artist_names.append(artist['name'])
                
                
    return all_artist_names
            
            
    
    

In [4]:
 playlist_id = '37i9dQZF1DX4o1oenSJRJd'

In [5]:
 artists_spot(playlist_id)

['Coldplay',
 'Eminem',
 'Kanye West',
 'Linkin Park',
 'The Killers',
 'Shakira',
 'The White Stripes',
 'Daddy Yankee',
 'Coldplay',
 'Linkin Park',
 'Eminem',
 'Kanye West',
 'Nelly Furtado',
 'Rihanna',
 'Coldplay',
 'Gorillaz',
 'Evanescence',
 'Eminem',
 'Foster The People',
 'Train',
 'Akon',
 'JAY-Z',
 'Coldplay',
 'USHER',
 'Kings of Leon',
 'Kesha',
 'Outkast',
 'Red Hot Chili Peppers',
 'Timbaland',
 'Jimmy Eat World',
 'Kanye West',
 'Lady Gaga',
 'Coldplay',
 'The Game',
 'Eminem',
 'Hoobastank',
 'Outkast',
 'Papa Roach',
 'Sean Paul',
 'Snow Patrol',
 'Nelly Furtado',
 'Maroon 5',
 'Nickelback',
 'Flo Rida',
 'Kendrick Lamar',
 'Kanye West',
 'Dr. Dre',
 'Nelly Furtado',
 'Black Eyed Peas',
 'Alicia Keys',
 '50 Cent',
 'Empire of the Sun',
 'Britney Spears',
 'The Fray',
 'Rihanna',
 'Kanye West',
 'Maroon 5',
 'Avril Lavigne',
 'Franz Ferdinand',
 'Justin Timberlake',
 'Dr. Dre',
 'Amy Winehouse',
 'Corinne Bailey Rae',
 'Shaggy',
 'Beyoncé',
 'Green Day',
 'Foo Fighter

In [6]:
#after we have our list of names
def get_songs_by_artists(playlist_id):
    artist_names = artists_spot(playlist_id)
    all_songs = []

    for artist in artist_names:
        #for every artist from playlist i will search for 40 tracks from artist
        results = sp.search(q='artist:' + artist, type='track', limit=40)
        for track in results['tracks']['items']:
            #appending the artist song and uri
            all_songs.append({
                "artist": artist,
                "song_name": track['name'],
                "uri": track['uri']
            })
            

    df = pd.DataFrame(all_songs)
    return df

In [7]:
df= get_songs_by_artists(playlist_id)

In [8]:
df = df.drop_duplicates(subset='song_name', keep = 'first')
track_uris = df['uri'].str.split(':').str[-1].tolist()

In [9]:
df['artist'].value_counts()

artist
Kanye West             40
Eve                    40
Foster The People      40
Modjo                  39
My Chemical Romance    39
                       ..
Norah Jones            14
Kylie Minogue          14
Coldplay               14
Taio Cruz              12
Nelly Furtado          11
Name: count, Length: 92, dtype: int64

In [10]:
def get_combined_audio_features(track_uris, df):
    
    start = 0
    audio = pd.DataFrame()
    
    for stop in range(0, len(track_uris) + 100, 100):
        if start != stop:
            audio_features = pd.json_normalize(sp.audio_features(track_uris[start:stop]))
            audio = pd.concat([audio_features,audio])
            start = stop
    
    # Merge the audio features DataFrame with the song_info_df on the 'uri' column
    combined_df = pd.merge(df, audio, on='uri', how='right')
    
    return combined_df

In [11]:
combined_df = get_combined_audio_features(track_uris, df)

In [12]:
combined_df['song_name'].value_counts()

song_name
What It's Like                                          1
PRIDE.                                                  1
Sugar (feat. Wynter)                                    1
What A Night (feat. inverness) - Big Game Winner Mix    1
High Heels - Whistle While You Twerk                    1
                                                       ..
Family Affair                                           1
Be Without You - Kendu Mix                              1
Real Love                                               1
Mary Jane (All Night Long)                              1
Papercut                                                1
Name: count, Length: 2773, dtype: int64

### I want to to be able to update the dataframe by adding new playlists and run the same functions and then adding to the existing df

In [35]:
existing_df = pd.read_csv('spotify_csv/spotify_df.csv')

In [36]:
existing_df.drop('Unnamed: 0', axis=1, inplace=True)

In [37]:
def add_new_songs(new_playlist_id):
    

    song_df = get_songs_by_artists(new_playlist_id)
    
    song_df = song_df.drop_duplicates(subset='song_name', keep = 'first')
    track_uris = song_df['uri'].str.split(':').str[-1].tolist()
    
    combined_df = get_combined_audio_features(track_uris, song_df)
    
    
    
    return combined_df

In [38]:
new_playlist_id = input('Enter the new playlist ID: ')
combined_df = add_new_songs(new_playlist_id)

Enter the new playlist ID: 37i9dQZF1DXaKIA8E7WcJj


In [39]:
spot_df = pd.concat([existing_df, combined_df], ignore_index = True)
spot_df = spot_df.drop_duplicates(subset= 'song_name', keep = 'first')

In [40]:
spot_df

,artist,song_name,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Eve,What It's Like,spotify:track:6vUPPEN2V2lZ78b6DyEIjZ,0.679,0.557,0,-7.423,1,0.0281,0.090400,0.032400,0.0866,0.405,85.152,audio_features,6vUPPEN2V2lZ78b6DyEIjZ,https://api.spotify.com/v1/tracks/6vUPPEN2V2lZ...,https://api.spotify.com/v1/audio-analysis/6vUP...,303133,4
1,Eve,Kaikai Kitan,spotify:track:6y4GYuZszeXNOXuBFsJlos,0.584,0.911,5,-3.515,0,0.0900,0.011900,0.000000,0.1190,0.606,92.502,audio_features,6y4GYuZszeXNOXuBFsJlos,https://api.spotify.com/v1/tracks/6y4GYuZszeXN...,https://api.spotify.com/v1/audio-analysis/6y4G...,221426,4
2,Eve,Hey Lover,spotify:track:0bhwnn2xqnBUcMZDcXNuII,0.480,0.482,7,-11.450,0,0.0362,0.250000,0.000289,0.3450,0.684,128.160,audio_features,0bhwnn2xqnBUcMZDcXNuII,https://api.spotify.com/v1/tracks/0bhwnn2xqnBU...,https://api.spotify.com/v1/audio-analysis/0bhw...,144000,4
3,Eve,Father Of Mine,spotify:track:2hx4ptqsE8dboLH3NCLmaN,0.568,0.940,1,-5.401,1,0.0411,0.000431,0.000050,0.0779,0.738,97.096,audio_features,2hx4ptqsE8dboLH3NCLmaN,https://api.spotify.com/v1/tracks/2hx4ptqsE8db...,https://api.spotify.com/v1/audio-analysis/2hx4...,231400,4
4,Eve,I See Red,spotify:track:5MAK1nd8R6PWnle1Q1WJvh,0.509,0.448,4,-7.552,0,0.0357,0.007130,0.013700,0.2440,0.221,156.909,audio_features,5MAK1nd8R6PWnle1Q1WJvh,https://api.spotify.com/v1/tracks/5MAK1nd8R6PW...,https://api.spotify.com/v1/audio-analysis/5MAK...,230613,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10491,Tammi Terrell,He's The One I Love,spotify:track:75fAdigZDBaYg4OqAm8w4X,0.712,0.796,2,-5.468,1,0.0264,0.003530,0.031000,0.0640,0.964,112.683,audio_features,75fAdigZDBaYg4OqAm8w4X,https://api.spotify.com/v1/tracks/75fAdigZDBaY...,https://api.spotify.com/v1/audio-analysis/75fA...,158480,4
10492,Tammi Terrell,There Are Things,spotify:track:6hJUAUrzkymZ3QU5iMLcdd,0.696,0.618,1,-12.234,1,0.0315,0.259000,0.000000,0.0793,0.929,114.502,audio_features,6hJUAUrzkymZ3QU5iMLcdd,https://api.spotify.com/v1/tracks/6hJUAUrzkymZ...,https://api.spotify.com/v1/audio-analysis/6hJU...,190547,4
10494,Tammi Terrell,"Give In, You Just Can't Win",spotify:track:4Ngc2nwfoNmtBy5eeERywy,0.633,0.651,10,-3.123,0,0.0360,0.122000,0.000037,0.2940,0.655,116.531,audio_features,4Ngc2nwfoNmtBy5eeERywy,https://api.spotify.com/v1/tracks/4Ngc2nwfoNmt...,https://api.spotify.com/v1/audio-analysis/4Ngc...,159053,4
10495,Tammi Terrell,That's How It Is (Since You've Been Gone),spotify:track:0WZlNt8Kux7gJCB97nWxe4,0.654,0.710,5,-4.802,0,0.0299,0.173000,0.000002,0.1090,0.741,118.535,audio_features,0WZlNt8Kux7gJCB97nWxe4,https://api.spotify.com/v1/tracks/0WZlNt8Kux7g...,https://api.spotify.com/v1/audio-analysis/0WZl...,208133,4


In [41]:
spot_df.to_csv('spotify_csv/spotify_df.csv')

## While its not perfect i think it does a good job at fetching new songs and adding them to our df.

Steps for use:
start by reading in csv and then run the code, input the playlist uri, run and then save the csv.
you need to read in the csv again to start the loop again.